# Data Acquisistion

* HTTP (*HyperText Transfer Protocol*): Plain text transportation
    * Request
    * Response
* HTML (*HyperText Markup Language*): Document structure for a webpage
* JSON (*JavaScript Object Notation*): Data interchange format based on JavaScript (structure is very similiar to Python dictionaries)
* API (*Application Programming Interface*): How things are interacted with programatically
* REST (*Representational State Transfer*): A set of rules for application urls

| HTTP Method | Endpoint         | Description                |
| ---         | ---              | ---                        |
| GET         | /{resource}/{id} | Read details of a resource |
| GET         | /{resource}      | A listing of resources     |
| POST        | /{resource}      | Create a new resource      |
| PATCH       | /{resource}/{id} | Update a resource          |
| DELETE      | /{resource}/{id} | Delete a resource          |

We'll focus on the GET methods as they are the ones that retrieve and let us read information.

## Imports

In [1]:
import pandas as pd
import requests

Example HTML pages:

- http://example.com
- https://alumni.codeup.com

## Using requests library

We get responses from making a requests.

### Some things we can get:

#### Response Code

In [2]:
response = requests.get('http://example.com')
response

<Response [200]>

Http status codes:

* 200s: everythings good
* 300s: redirecting
* 400s: you did something wrong
* 500s: something is wrong with the server

#### Response Text

In [3]:
print(response.text)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

### Example JSON API endpoints:

* https://aphorisms.glitch.me
* https://jsonplaceholder.typicode.com/posts/1
* https://jsonplaceholder.typicode.com/users

### Let's start with the quote generator

In [4]:
response = requests.get('https://aphorisms.glitch.me')
response

<Response [200]>

In [5]:
response.text

'{"quote":"You don\'t manage people; you manage things. You lead people.","author":"Admiral Grace Hopper"}'

In [6]:
data = response.json()
data

{'quote': "You don't manage people; you manage things. You lead people.",
 'author': 'Admiral Grace Hopper'}

#### What is the difference between `response.text` and `response.json()`?

In [7]:
print('response.text type is:', type(response.text))
print('response.json() type is:', type(response.json()))

response.text type is: <class 'str'>
response.json() type is: <class 'dict'>


#### So now we can treat `data` just like a dictionary?

In [8]:
data

{'quote': "You don't manage people; you manage things. You lead people.",
 'author': 'Admiral Grace Hopper'}

In [9]:
data['quote']

"You don't manage people; you manage things. You lead people."

In [10]:
data['author']

'Admiral Grace Hopper'

## Now let's work with an API storing time series data:

In [11]:
url = 'https://python.zgulde.net'
response = requests.get(url)
response.json()

{'api': '/api/v1', 'help': '/documentation'}

#### &#8593; This tells us what we can add to url to get new info

#### Let's look at the documentation

In [12]:
url = 'https://python.zgulde.net' + '/documentation'
response = requests.get(url)
response.json()

{'payload': '\nThe API accepts GET requests for all endpoints, where endpoints are prefixed\nwith\n\n    /api/{version}\n\nWhere version is "v1"\n\nValid endpoints:\n\n- /stores[/{store_id}]\n- /items[/{item_id}]\n- /sales[/{sale_id}]\n\nAll endpoints accept a `page` parameter that can be used to navigate through\nthe results.\n',
 'status': 'ok'}

In [13]:
response.json()['payload']

'\nThe API accepts GET requests for all endpoints, where endpoints are prefixed\nwith\n\n    /api/{version}\n\nWhere version is "v1"\n\nValid endpoints:\n\n- /stores[/{store_id}]\n- /items[/{item_id}]\n- /sales[/{sale_id}]\n\nAll endpoints accept a `page` parameter that can be used to navigate through\nthe results.\n'

In [14]:
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



#### What's an endpoint?

##### * An endpoint are the parts after the main url, called the domain. 

##### * In this case our endpoints go after .com in the url separated by slashes.

   * **Extra**: .com, .gov, .net are known as TLD or *Top Level Domains* in a url

#### So with this info we can now start retrieving data from the api

#### Let's check out the stores data

In [15]:
url = 'https://python.zgulde.net/api/v1/stores'
response = requests.get(url)
data = response.json()
data

{'payload': {'max_page': 1,
  'next_page': None,
  'page': 1,
  'previous_page': None,
  'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
    'store_city': 'San Antonio',
    'store_id': 1,
    'store_state': 'TX',
    'store_zipcode': '78253'},
   {'store_address': '9255 FM 471 West',
    'store_city': 'San Antonio',
    'store_id': 2,
    'store_state': 'TX',
    'store_zipcode': '78251'},
   {'store_address': '2118 Fredericksburg Rdj',
    'store_city': 'San Antonio',
    'store_id': 3,
    'store_state': 'TX',
    'store_zipcode': '78201'},
   {'store_address': '516 S Flores St',
    'store_city': 'San Antonio',
    'store_id': 4,
    'store_state': 'TX',
    'store_zipcode': '78204'},
   {'store_address': '1520 Austin Hwy',
    'store_city': 'San Antonio',
    'store_id': 5,
    'store_state': 'TX',
    'store_zipcode': '78218'},
   {'store_address': '1015 S WW White Rd',
    'store_city': 'San Antonio',
    'store_id': 6,
    'store_state': 'TX',
    'store_zipcode': '78220

In [16]:
data.keys()

dict_keys(['payload', 'status'])

In [17]:
data['status']

'ok'

In [18]:
data['payload']

{'max_page': 1,
 'next_page': None,
 'page': 1,
 'previous_page': None,
 'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
   'store_city': 'San Antonio',
   'store_id': 1,
   'store_state': 'TX',
   'store_zipcode': '78253'},
  {'store_address': '9255 FM 471 West',
   'store_city': 'San Antonio',
   'store_id': 2,
   'store_state': 'TX',
   'store_zipcode': '78251'},
  {'store_address': '2118 Fredericksburg Rdj',
   'store_city': 'San Antonio',
   'store_id': 3,
   'store_state': 'TX',
   'store_zipcode': '78201'},
  {'store_address': '516 S Flores St',
   'store_city': 'San Antonio',
   'store_id': 4,
   'store_state': 'TX',
   'store_zipcode': '78204'},
  {'store_address': '1520 Austin Hwy',
   'store_city': 'San Antonio',
   'store_id': 5,
   'store_state': 'TX',
   'store_zipcode': '78218'},
  {'store_address': '1015 S WW White Rd',
   'store_city': 'San Antonio',
   'store_id': 6,
   'store_state': 'TX',
   'store_zipcode': '78220'},
  {'store_address': '12018 Perrin Beitel 

In [19]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [20]:
data['payload']['stores']

[{'store_address': '12125 Alamo Ranch Pkwy',
  'store_city': 'San Antonio',
  'store_id': 1,
  'store_state': 'TX',
  'store_zipcode': '78253'},
 {'store_address': '9255 FM 471 West',
  'store_city': 'San Antonio',
  'store_id': 2,
  'store_state': 'TX',
  'store_zipcode': '78251'},
 {'store_address': '2118 Fredericksburg Rdj',
  'store_city': 'San Antonio',
  'store_id': 3,
  'store_state': 'TX',
  'store_zipcode': '78201'},
 {'store_address': '516 S Flores St',
  'store_city': 'San Antonio',
  'store_id': 4,
  'store_state': 'TX',
  'store_zipcode': '78204'},
 {'store_address': '1520 Austin Hwy',
  'store_city': 'San Antonio',
  'store_id': 5,
  'store_state': 'TX',
  'store_zipcode': '78218'},
 {'store_address': '1015 S WW White Rd',
  'store_city': 'San Antonio',
  'store_id': 6,
  'store_state': 'TX',
  'store_zipcode': '78220'},
 {'store_address': '12018 Perrin Beitel Rd',
  'store_city': 'San Antonio',
  'store_id': 7,
  'store_state': 'TX',
  'store_zipcode': '78217'},
 {'store

In [21]:
pd.DataFrame(data['payload']['stores'])

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


#### Let's do the same with items

In [22]:
url = 'https://python.zgulde.net/api/v1/items'
response = requests.get(url)
data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [23]:
data['status']

'ok'

In [24]:
data['payload']

{'items': [{'item_brand': 'Riceland',
   'item_id': 1,
   'item_name': 'Riceland American Jazmine Rice',
   'item_price': 0.84,
   'item_upc12': '35200264013',
   'item_upc14': '35200264013'},
  {'item_brand': 'Caress',
   'item_id': 2,
   'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
   'item_price': 6.44,
   'item_upc12': '11111065925',
   'item_upc14': '11111065925'},
  {'item_brand': 'Earths Best',
   'item_id': 3,
   'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
   'item_price': 2.43,
   'item_upc12': '23923330139',
   'item_upc14': '23923330139'},
  {'item_brand': 'Boars Head',
   'item_id': 4,
   'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
   'item_price': 3.14,
   'item_upc12': '208528800007',
   'item_upc14': '208528800007'},
  {'item_brand': 'Back To Nature',
   'item_id': 5,
   'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
   'item_price': 2.61,
   'item_upc12': '759283100036',

In [26]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [27]:
(
    data['payload']['page'], 
    data['payload']['max_page'], 
    data['payload']['next_page'],
    data['payload']['previous_page'],
)

(1, 3, '/api/v1/items?page=2', None)

In [28]:
pd.DataFrame(data['payload']['items'])

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


### Now what to do about the multiple pages?

In [29]:
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/items'
items = []

url = domain + endpoint

response = requests.get(url)
data = response.json()
# .extend adds elemnts from a list to another list
items.extend(data['payload']['items'])

In [30]:
items

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [31]:
data['payload']['next_page']

'/api/v1/items?page=2'

In [32]:
url = domain + data['payload']['next_page']
print('Next url:', url)

Next url: https://python.zgulde.net/api/v1/items?page=2


In [33]:
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [34]:
items

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [35]:
url = domain + data['payload']['next_page']
print('next url:', url)

next url: https://python.zgulde.net/api/v1/items?page=3


In [36]:
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [37]:
items

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [38]:
# Hint hint: if data['payload']['next_page'] is None:
print('next endpoint', data['payload']['next_page'])

next endpoint None


In [39]:
pd.DataFrame(items)
# next steps:
# save to a csv or wrap up everything in a function

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


In [40]:
# setup
domain = 'https://python.zgulde.net'
endpoint = '/api/v1/items'
items = []

# For each page -- until next page is None
url = domain + endpoint
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])
# update the endpoint
endpoint = data['payload']['next_page']

## Guidance for the exercise
1. Setup
    * url (base + endpoint)
    * empty list
1. Loop
    1. make a request
    1. handle the response, add to the list
    1. find the next url endpoint
        1. if it's None, stop looping
        1. if it's a string, use it to construct the next url
1. Turn the list into a dataframe

### General Tips

![Alt Text](https://c.tenor.com/NpDMsR4GdTAAAAAC/salute-captain-america.gif)

* solve an easy problem first (the items endpoint), then apply that solution to the larger problem (sales)
* informational print statements are helpful as you are developing code, especially inside of a loop to see what changes
* Dont' be afraid to command + shift + p (command + shift + c for jupyter lab) "interrupt the kernel"

# Exercises

Create a new local git repository and remote repository on github named `time-series-exercises`. Save this work for this module in your `time-series-exercises` repo.

The end result of this exercise should be a file named `acquire.py`.

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named `items` that has all of the data for items.
2. Do the same thing, but for `stores` (https://python.zgulde.net/api/v1/stores)
3. Extract the data for `sales` (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.
4. Save the data in your files to local csv files so that it will be faster to access in the future.
5. Combine the data from your three separate dataframes into one large dataframe.
6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv
7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the `acquire.py` file and be able to re-run the functions and get the same data.